In [27]:
%pip install pymysql sqlalchemy
%pip install cryptography


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\ADMIN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached cffi-2.0.0-cp312-cp312-win_amd64.whl.metadata (2.6 kB)
  Using cached pycparser-2.23-py3-none-any.whl.metadata (993 bytes)
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   -------- ------------------------------- 0.8/3.5 MB 5.6 MB/s eta 0:00:01
   -------------------- ------------------- 1.8/3.5 MB 5.3 MB/s eta 0:00:01
   ----------------------------- ---------- 2.6/3.5 MB 4.6 MB/s eta 0:00:01
   -------------------------------------- - 3.4/3.5 MB 4.4 MB/s eta 0:00:01
   ---------------------------------------- 3.5/3.5 MB 4.2 MB/s  0:00:00
Using cached cffi-2.0.0-cp312-cp312-win_amd64.whl (183 kB)
Using cached pycparser-2.23-py3-none-any.whl (118 kB)

   ---------------------------------------- 0/3 [pycparser]
   ---------------------------------------- 0/3 [pycparser]
   ---------------------------------------- 0/3 [pycparser]
   ---------------------------------------- 


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\ADMIN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import mysql.connector
import pymysql
from sqlalchemy import create_engine

In [2]:
df=pd.read_csv("Cleaned_Luxury_Housing_Bangalore.csv")
df.head()

,Property_ID,Micro_Market,Project_Name,Developer_Name,Unit_Size_Sqft,Configuration,Ticket_Price_Cr,Transaction_Type,Buyer_Type,Purchase_Quarter,...,Sales_Channel,NRI_Buyer,Locality_Infra_Score,Avg_Traffic_Time_Min,Buyer_Comments,Price_per_Sqft,Quarter_Label,Booking_Status,Booking_Count,Booking_Conversion_Rate
0,PROP000001,Sarjapur Road,Project_0,Rmz,4025.0,4BHK,12.750846,Primary,NRI,2025-03-31,...,Broker,1,9.212491,18,Loved The Amenities!,31679.120594,2025Q1,Booked,1,0.494
1,PROP000002,Indiranagar,Project_1,Puravankara,5760.0,3BHK,16.292152,Primary,Other,2024-06-30,...,Nri Desk,0,7.723898,106,No Comments,28284.985887,2024Q2,Booked,1,0.508
2,PROP000003,Bannerghatta Road,Project_2,Tata Housing,7707.0,4BHK,10.517724,Primary,HNI,2023-12-31,...,Direct,1,6.985493,113,Agent Was Not Responsive.,13646.976013,2023Q4,Booked,1,0.506
3,PROP000004,Bellary Road,Project_3,Embassy,6192.0,3BHK,9.396367,Primary,HNI,2024-03-31,...,Online,1,6.100929,106,Excellent Location!,15175.012103,2024Q1,Booked,1,0.497
4,PROP000005,Koramangala,Project_4,Snn Raj,7147.0,4BHK,15.345392,Secondary,HNI,2024-12-31,...,Broker,0,5.312510,18,Too Far From My Office.,21471.096187,2024Q4,Not Booked,0,0.507


In [3]:
from sqlalchemy import create_engine

# MySQL connection details
username = "root"
password = "123456789"
host = "localhost"
database = "luxury_housing"

# Create SQLAlchemy engine using mysql+mysqlconnector
engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}/{database}")
with engine.connect() as connection:
    print("Connection to MySQL database was successful!")

Connection to MySQL database was successful!


In [5]:
df.to_sql(
    name="luxury_housing_cleaned",
    con=engine,
    if_exists="replace",
    index=False)

print("Data loaded into MySQL successfully!")


Data loaded into MySQL successfully!


case_1

In [6]:
# Top 10 micro-markets by total sales value
market_perf = df.groupby('Micro_Market')['Ticket_Price_Cr'].sum().sort_values(ascending=False).head(10)

# Average ticket price per developer
builder_perf = df.groupby('Developer_Name')['Ticket_Price_Cr'].mean().sort_values(ascending=False).head(10)

# Configuration demand (2BHK, 3BHK, etc.)
config_demand = df['Configuration'].value_counts()


case2

In [7]:
sales_perf = df.groupby('Possession_Status')['Ticket_Price_Cr'].agg(['count','mean'])
sales_perf

,count,mean
Possession_Status,,
Launch,33641,12.661950
Ready To Move,33590,12.625654
Under Construction,33769,12.639052


case3

In [8]:
# Buyer Type Distribution
buyer_dist = df['Buyer_Type'].value_counts(normalize=True) * 100

# Simple sentiment check (optional if Buyer_Comments are usable)
df['Positive_Flag'] = df['Buyer_Comments'].str.contains('good|excellent|amazing|spacious', case=False, na=False).astype(int)
sentiment_summary = df.groupby('Buyer_Type')['Positive_Flag'].mean()


In [9]:
sentiment_summary

Buyer_Type
CXO                0.089331
HNI                0.091108
NRI                0.092344
Other              0.095141
Startup Founder    0.089819
Name: Positive_Flag, dtype: float64

case4

In [10]:
pricing_trends = df.groupby('Developer_Name')['Ticket_Price_Cr'].agg(['mean','median','count']).sort_values(by='mean', ascending=False).head(10)
pricing_trends

,mean,median,count
Developer_Name,,,
Sobha,12.787157,12.016263,9043
Total Environment,12.738714,12.017281,9209
L&T Realty,12.703113,12.029339,9225
Godrej,12.680854,12.009541,9215
Rmz,12.663454,12.017281,9092
Puravankara,12.655817,12.000000,9166
Prestige,12.635095,12.000000,9318
Tata Housing,12.597294,12.023037,9019
Snn Raj,12.539817,12.000000,9309


case5

In [11]:
df[['Amenity_Score', 'Ticket_Price_Cr']].corr()


,Amenity_Score,Ticket_Price_Cr
Amenity_Score,1.000000,0.001368
Ticket_Price_Cr,0.001368,1.000000


case6

In [12]:
quarter_trend = df.groupby('Quarter_Label')['Ticket_Price_Cr'].sum().reset_index()
quarter_trend

,Quarter_Label,Ticket_Price_Cr
0,2023Q2,160515.342725
1,2023Q3,158963.179944
2,2023Q4,159699.757336
3,2024Q1,159962.306111
4,2024Q2,160095.532068
5,2024Q3,160136.612685
6,2024Q4,159059.544546
7,2025Q1,158432.261726


In [13]:
df.head(10)

,Property_ID,Micro_Market,Project_Name,Developer_Name,Unit_Size_Sqft,Configuration,Ticket_Price_Cr,Transaction_Type,Buyer_Type,Purchase_Quarter,...,NRI_Buyer,Locality_Infra_Score,Avg_Traffic_Time_Min,Buyer_Comments,Price_per_Sqft,Quarter_Label,Booking_Status,Booking_Count,Booking_Conversion_Rate,Positive_Flag
0,PROP000001,Sarjapur Road,Project_0,Rmz,4025.0,4BHK,12.750846,Primary,NRI,2025-03-31,...,1,9.212491,18,Loved The Amenities!,31679.120594,2025Q1,Booked,1,0.494,0
1,PROP000002,Indiranagar,Project_1,Puravankara,5760.0,3BHK,16.292152,Primary,Other,2024-06-30,...,0,7.723898,106,No Comments,28284.985887,2024Q2,Booked,1,0.508,0
2,PROP000003,Bannerghatta Road,Project_2,Tata Housing,7707.0,4BHK,10.517724,Primary,HNI,2023-12-31,...,1,6.985493,113,Agent Was Not Responsive.,13646.976013,2023Q4,Booked,1,0.506,0
3,PROP000004,Bellary Road,Project_3,Embassy,6192.0,3BHK,9.396367,Primary,HNI,2024-03-31,...,1,6.100929,106,Excellent Location!,15175.012103,2024Q1,Booked,1,0.497,1
4,PROP000005,Koramangala,Project_4,Snn Raj,7147.0,4BHK,15.345392,Secondary,HNI,2024-12-31,...,0,5.312510,18,Too Far From My Office.,21471.096187,2024Q4,Not Booked,0,0.507,0
5,PROP000006,Yelahanka,Project_5,Brigade,4290.0,4BHK,9.800000,Secondary,HNI,2023-09-30,...,0,5.154027,16,Underpriced For Location.,22843.822844,2023Q3,Not Booked,0,0.489,0
6,PROP000007,Bannerghatta Road,Project_6,Prestige,6776.0,5BHK+,10.896752,Secondary,Other,2025-03-31,...,0,9.791550,74,Too Far From My Office.,16081.393675,2025Q1,Not Booked,0,0.506,0
7,PROP000008,Whitefield,Project_7,Total Environment,5763.0,3BHK,9.357539,Secondary,NRI,2025-03-31,...,1,9.054713,31,No Comments,16237.270421,2025Q1,Not Booked,0,0.498,0
8,PROP000009,Sarjapur Road,Project_8,Prestige,5815.0,3BHK,12.299449,Secondary,Other,2024-03-31,...,1,8.552046,68,No Comments,21151.245761,2024Q1,Not Booked,0,0.494,0
9,PROP000010,Jp Nagar,Project_9,Puravankara,8675.0,4BHK,9.193310,Primary,CXO,2023-09-30,...,1,8.395691,18,Too Far From My Office.,10597.475826,2023Q3,Booked,1,0.502,0


In [14]:
df["Transaction_Type"].unique()

array(['Primary', 'Secondary'], dtype=object)

In [15]:
df.columns

Index(['Property_ID', 'Micro_Market', 'Project_Name', 'Developer_Name',
       'Unit_Size_Sqft', 'Configuration', 'Ticket_Price_Cr',
       'Transaction_Type', 'Buyer_Type', 'Purchase_Quarter',
       'Connectivity_Score', 'Amenity_Score', 'Possession_Status',
       'Sales_Channel', 'NRI_Buyer', 'Locality_Infra_Score',
       'Avg_Traffic_Time_Min', 'Buyer_Comments', 'Price_per_Sqft',
       'Quarter_Label', 'Booking_Status', 'Booking_Count',
       'Booking_Conversion_Rate', 'Positive_Flag'],
      dtype='object')